# Convolutional Neural Networks

We will now build toward the next big idea in neural networks: convolutional neural networks (frequently called CNNs or ConvNets). These are nets that take some inspiration from the way neurons are connected in an animal's visual cortex, where different groups of neurons respond to specific portions of the visual field and then those signals are combined in a sort of hierarchy that is thought to allow animals to extract increasingly complex features within the incoming visual stimuli.

CNNs are are feedforward neural networks like we have seen, but neurons in one layer are not necessarily connected to every neuron in the next layer. Such a more complex architecture, in some sense, allows a regularized version of fully-connected nets in that, they can generalize better to test data and to the real world. Recall, we could learn the whole training set for CIFAR-10 but had at least a 30% less accuracy on test data. Another benefit is that this more sparse sort of structure means we can build deeper or wider neural nets without growing the computation as much as fully-connected nets.

CNNs are some of the best algorithms for image and video recognition problems, but have also been used effectively in anomaly detection, time series analysis in financial markets, and predicting the interactions between proteins and molecules in drug discovery among other applications.

(Please see the class notes for introductory material. We continue with implementing some CNNs.)

As we have seen, implementing CNNs is not overly complex, as there is a version of backpropagation that will work and the feature extractors in convolutional layers work fairly simply. However, CNNs are generally pretty expensive to run, so in the interest of having optimized code, we will use Keras code for our CNNs in the class.

## ShallowNet

Let's make a small CNN called ShallowNet just to get a sense of how to use Keras to build one. First, import some things from Keras.

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import plot_model

Next, let's create a class for the net.

In [32]:
class ShallowNet:
    # create the architecture
    def build(height, width, depth, numFilters, classes):
        # create a feedforward neural net
        model = Sequential()
        
        # add a convolutional layer with numFilters number of 3x3 filters
        model.add(Conv2D(numFilters, (3, 3), padding='same', input_shape = (height, width, depth)))
        model.add(Activation('relu'))
        
        # add a softmax classifier
        model.add(Flatten())
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

Next, let's try to run it on MNIST

In [56]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [55]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)

trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

model = ShallowNet.build(28, 28, 1, 32, 10)
model.compile(loss = 'categorical_crossentropy', optimizer = SGD(0.01), metrics = ['accuracy'])

print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_split = 0.20, batch_size = 32, epochs = 100, verbose = 1)

print('\n Test accuracy')
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)
testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY))

[INFO] training network...
Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 30s 633us/sample - loss: 0.4778 - accuracy: 0.8722 - val_loss: 0.3005 - val_accuracy: 0.9149
Epoch 2/100
48000/48000 [==============================] - 29s 597us/sample - loss: 0.3100 - accuracy: 0.9105 - val_loss: 0.2815 - val_accuracy: 0.9202
Epoch 3/100
48000/48000 [==============================] - 29s 595us/sample - loss: 0.2827 - accuracy: 0.9194 - val_loss: 0.2637 - val_accuracy: 0.9250
Epoch 4/100
48000/48000 [==============================] - 28s 574us/sample - loss: 0.2555 - accuracy: 0.9280 - val_loss: 0.2292 - val_accuracy: 0.9367
Epoch 5/100
48000/48000 [==============================] - 28s 588us/sample - loss: 0.2256 - accuracy: 0.9373 - val_loss: 0.2018 - val_accuracy: 0.9463
Epoch 6/100
48000/48000 [==============================] - 29s 600us/sample - loss: 0.1967 - accuracy: 0.9444 - val_loss: 0.1799 - val_accuracy: 0.9516
Epoch 7/100

In [ ]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)

trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

model = ShallowNet.build(28, 28, 1, 32, 10)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_split = 0.20, batch_size = 32, epochs = 100, verbose = 1)

print('\n Test accuracy')
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)
testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY))

## LeNet



In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

class LeNet:
    # create the architecture
    def build(height, width, depth, classes):
        # create a feedforward neural net
        model = Sequential()
        
        # add a convolutional layer with 20 5x5 filters and a 2x2 max pooling layer
        model.add(Conv2D(20, (5, 5), padding = 'same', input_shape = (height, width, depth), activation = 'tanh'))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        
        # add another convolutional layer with 50 5x5 filters and a 2x2 max pooling layer
        model.add(Conv2D(50, (5, 5), padding='same'))
        model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
        
        # add a fully-connected layer
        model.add(Flatten())
        model.add(Dense(500, activation = 'tanh'))
        
        # add a softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

Let's run the model on MNIST.

In [ ]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)

trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

model = LeNet.build(28, 28, 1, 10)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_split = 0.20, batch_size = 128, epochs = 20, verbose = 1)

print('\n Test accuracy')
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)
testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY))

[INFO] training network...
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 45s 937us/sample - loss: 0.1791 - accuracy: 0.9455 - val_loss: 0.0649 - val_accuracy: 0.9805
Epoch 2/20
28672/48000 [================>.............] - ETA: 17s - loss: 0.0574 - accuracy: 0.9823